## Pegar dados e dissolver CEPs

In [ ]:
import pandas as pd
import requests
import os
import time

# Define o caminho do arquivo da planilha
caminho_planilha = r'C:\Users\Usuario\Desktop\arquivo1.xlsx'

# Carrega a planilha usando o pandas
df = pd.read_excel(caminho_planilha)

# Obtém o número de linhas antes da dissolução
num_linhas_antes = df.shape[0]

# Obtém os valores únicos do campo CEP
cep_unicos = df['CEP'].unique()

# Obtém o número de linhas depois da dissolução
num_linhas_depois = len(cep_unicos)

# Imprime o número de linhas antes e depois da dissolução
print(f"Número de linhas antes da dissolução: {num_linhas_antes}")
print(f"Número de linhas depois da dissolução: {num_linhas_depois}")

# Dataset
csv_dissolved = pd.DataFrame({'CEP': cep_unicos})

## Converter CEP para Lat e Long

In [ ]:
# Função para obter latitude e longitude do CEP usando a API do MapQuest
def obter_latitude_longitude(cep, chave_api):
    url = f"http://www.mapquestapi.com/geocoding/v1/address?key={chave_api}&location={cep}"
    response = requests.get(url)
    data = response.json()
    
    if response.status_code == 200:
        results = data["results"][0]["locations"][0]["latLng"]
        latitude = results["lat"]
        longitude = results["lng"]
        return latitude, longitude
    else:
        return None, None

# Ler a planilha usando pandas
planilha = csv_dissolved

# Converter CEPs para latitude e longitude
planilha["Latitude"] = ""
planilha["Longitude"] = ""

# Definir o número de linhas por lote
linhas_por_lote = 100

# Percorrer as linhas em lotes
for i in range(0, len(planilha), linhas_por_lote):
    # Obter as linhas do lote atual
    lote = planilha.iloc[i:i+linhas_por_lote]

    # Processar cada linha do lote
    for index, row in lote.iterrows():
        cep = row["CEP"]
        latitude, longitude = obter_latitude_longitude(cep, "dSnLUo3ZbQ7hfKPASUdmN1iQuFl59Sbc")
        planilha.at[index, "Latitude"] = latitude
        planilha.at[index, "Longitude"] = longitude

    # Aguardar alguns segundos antes do próximo lote
    time.sleep(5)  # Aguarda 5 segundos

# Dataset final    
dataset = planilha

print("Convertido com sucesso")


## Verificar caso ele encontrou alguma coordenada incorreta

In [ ]:
# Converter a coluna "Latitude" para o tipo numérico (float)
dataset.loc[:, 'Latitude'] = dataset['Latitude'].astype(float)

# Contar o número de linhas do DataFrame original
total_linhas_original = dataset.shape[0]

# Filtrar as linhas em que os valores da coluna "Latitude" são negativos
dataset = dataset.loc[dataset['Latitude'] < 0]

# Contar o número de linhas do DataFrame filtrado
total_linhas_filtradas = dataset.shape[0]

# Dataset filtrado
resultado = dataset

# Exibir o resultado
print("Foram encontrado e deletados {} coordenadas incorretas".format(total_linhas_original - total_linhas_filtradas))

## Juntar resultado com a planilha completa

In [ ]:
# Juntar resultado com a planilha completa
planilha_completa = df.merge(resultado, on='CEP', how='left')

# Salvar a planilha completa com as novas colunas de latitude e longitude na área de trabalho
caminho_salvar = os.path.expanduser("~/Desktop/nova_planilha.xlsx")
planilha_completa.to_excel(caminho_salvar, index=False)

print(f"A nova planilha foi salva em: {caminho_salvar}")